In [1]:
import findspark
findspark.init()
import re
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
def clean_tweet(row):
    split_row=row.split(",")
    sentiment=split_row[0]
    tweet=split_row[5]
    tweet=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    words_list=tweet.split()
    return(sentiment,split_row[1],split_row[2],split_row[3],split_row[4] , words_list)

In [3]:
sc=SparkContext(appName='classifier')    
spark = SparkSession.builder.master('local[4]').config("spark.driver.memory","16g").config("spark.executor.memoryOverhead","1000M").appName('classifier').getOrCreate()
data=sc.textFile("trainingandtestdata/training.1600000.processed.noemoticon.csv")
data2=sc.textFile("trainingandtestdata/testdata.manual.2009.06.14.csv")
df=data.map(lambda r: clean_tweet(r))
df2=data2.map(lambda r: clean_tweet(r))

In [5]:
dataframe=spark.createDataFrame(df,['polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet'])
dataframe2=spark.createDataFrame(df2,['polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet'])

In [6]:
tem=dataframe.union(dataframe2)

In [7]:
from pyspark.ml.feature import StopWordsRemover
remover=StopWordsRemover(inputCol="Tweet", outputCol="filtered")
tem=remover.transform(tem)

In [8]:
from pyspark.ml.feature import NGram
ngram=NGram(n=2, inputCol="filtered", outputCol="2gram")
tem=ngram.transform(tem)
from pyspark.ml.feature import HashingTF,IDF
from pyspark.sql.types import IntegerType
hashingtf=HashingTF(inputCol="2gram", outputCol="tf", numFeatures=20000)
tem=hashingtf.transform(tem)
tem= tem.withColumn("polarity", tem["polarity"].cast(IntegerType()))
tem=tem.rdd

In [9]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import  Vectors as MLLibVectors
train_dataset=tem.map(lambda x: LabeledPoint(float(x.polarity), MLLibVectors.fromML(x.tf)))
training, test=train_dataset.randomSplit([0.8, 0.2])

In [15]:
from pyspark.mllib.classification import NaiveBayes
model=NaiveBayes.train(training, 1.0)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8) (192.168.1.227 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 611, in process
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\rdd.py", line 1562, in takeUpToNumLeft
    yield next(iterator)
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\rddsampler.py", line 95, in func
    for obj in iterator:
 